# Running Locally 

In [4]:
# working with ollama running locally
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama


# model name to use
model_name = "deepseek-r1:8b"

# question to ask the model
question = "What is the capital of France?"

# create a chat model chat template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{question}")
    ]
)

llm = ChatOllama(model=model_name)


result = llm.invoke(prompt.format_prompt(question=question).to_messages())

print(result)




C:\Users\Dhung\AppData\Local\Temp\ipykernel_9388\3920444572.py:20: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=model_name)


content='<think>\nOkay, the user asked: "What is the capital of France?" Let me start by recalling that Paris is indeed the capital. But I should make sure there\'s no chance they\'re thinking of another city. Maybe they heard something different or have outdated info.\n\nThey could be a student studying geography for a test, someone planning a trip who needs to know where to go in France, or just curious. Since it\'s a straightforward question, the answer is Paris. But I should consider if there\'s more context needed here. The user might want additional information like why Paris is the capital, its significance, or related facts about France.\n\nAlso, checking for any possible confusion with other cities that are sometimes mistaken as capitals—like maybe Toulouse? No, Toulouse isn\'t a capital at all. Lyon is important but not the capital. So confirming that Paris is correct and addressing potential misunderstandings could be helpful. \n\nIncluding key points such as its historical 